In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopy
import nominatim
import requests

derecho911_df = pd.read_csv('../data/911_Phone Calls_Derecho_050320 050420.csv')
tornado911_df = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
derecho_incidents_df = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv', skiprows=3)
tornado_incidents_df = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows=3)
single_family_housing_df = pd.read_csv('../data/2017 single family_final_with_sqft.txt')

In [17]:
derecho_incidents_df['IncidentTypeDescription1'].value_counts()
derecho911_df.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE


In [18]:
##print(derecho_incidents_df['IncidentDate'])
single_family_housing_df['2017 TOTAL ASSD'].max()

2888680

In [27]:
derech_nans = derecho911_df['ALI Latitude'].isna()
derecho911_nans_df = derecho911_df[derech_nans]
print(derecho911_nans_df)

     Seizure DateTime CallTypeId  ALI Latitude  ALI Longitude  \
49     5/3/2020 16:39  911 Calls           NaN            NaN   
53     5/3/2020 16:41  911 Calls           NaN            NaN   
56     5/3/2020 16:41  911 Calls           NaN            NaN   
141    5/3/2020 16:59  911 Calls           NaN            NaN   
142    5/3/2020 17:00  911 Calls           NaN            NaN   
...               ...        ...           ...            ...   
1435   5/3/2020 22:31  911 Calls           NaN            NaN   
1470   5/3/2020 22:54  911 Calls           NaN            NaN   
1472   5/3/2020 22:55  911 Calls           NaN            NaN   
1499   5/3/2020 23:17  911 Calls           NaN            NaN   
1548    5/4/2020 0:29  911 Calls           NaN            NaN   

         Cell Tower Address  
49                      NaN  
53                      NaN  
56                      NaN  
141                     NaN  
142                     NaN  
...                     ...  
1435     

In [64]:
HOST = "https://nominatim.openstreetmap.org/search?q="
HOST = HOST + '712 Main Street'
HOST = HOST + ',Nashville,Tennessee&format=json&polygon=1&addressdetails=1'
req = requests.get(HOST)
req_dict = req.json()
print(req_dict[0]['address']['postcode'])
##df2 = pd.DataFrame(columns=col_names, data=req.json()[1:])
##df2.head()

37206


In [58]:
pd.Series(data=req.json()[:])

0    {'place_id': 266399049, 'licence': 'Data © Ope...
dtype: object


In [20]:
derech_not_nans = derecho911_df['ALI Latitude'].notna()
derecho911_df = derecho911_df[derech_not_nans]

In [21]:
derecho911_df.shape

(1483, 5)

In [ ]:
derecho911_df['ALI Latitude'] = derecho911_df['ALI Latitude'].fillna(0)
derecho911_df['ALI Longitude'] = derecho911_df['ALI Longitude'].fillna(0)
##Turn NaNs into zeroes
derecho911_df['LatLong'] = derecho911_df.apply(lambda row: [row['ALI Latitude'], row['ALI Longitude']], axis=1)
##Make a latLong column which contains an array which is the form required for map markers

In [ ]:
derech_map = folium.Map(location=[36.247802, -86.716847], zoom_start=10)
##This is an arbitrary location within Nashville to start; this could be more centered

In [ ]:
folium.TileLayer('stamentoner').add_to(derech_map)

In [ ]:
for location in derecho911_df['LatLong']:
    folium.Marker(location=location).add_to(derech_map)

In [ ]:
map

In [ ]:
tornado911_df['ALI Latitude'] = tornado911_df['ALI Latitude'].fillna(0)
tornado911_df['ALI Longitude'] = tornado911_df['ALI Longitude'].fillna(0)
##Turn NaNs into zeroes
tornado911_df['LatLong'] = tornado911_df.apply(lambda row: [row['ALI Latitude'], row['ALI Longitude']], axis=1)
##Make a latLong column which contains an array which is the form required for map markers

In [ ]:
torn_map = folium.Map(location=[36.247802, -86.716847], zoom_start=10)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(torn_map)
for location in tornado911_df['LatLong']:
    folium.Marker(location=location).add_to(torn_map)
torn_map

In [ ]:
derecho_incidents_df['Latitude1'] = derecho_incidents_df['Latitude1'].fillna(0)
derecho_incidents_df['Longitude1'] = derecho_incidents_df['Longitude1'].fillna(0)
##Turn NaNs into zeroes
derecho_incidents_df['LatLong'] = derecho_incidents_df.apply(lambda row: [row['Latitude1'], row['Longitude1']], axis=1)
##Make a latLong column which contains an array which is the form required for map markers
derech_inc_map = folium.Map(location=[36.247802, -86.716847], zoom_start=10)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(derech_inc_map)
for location in derecho_incidents_df['LatLong']:
    folium.Marker(location=location).add_to(derech_inc_map)
derech_inc_map

In [ ]:
tornado_incidents_df['Latitude1'] = tornado_incidents_df['Latitude1'].fillna(0)
tornado_incidents_df['Longitude1'] = tornado_incidents_df['Longitude1'].fillna(0)
##Turn NaNs into zeroes
tornado_incidents_df['LatLong'] = tornado_incidents_df.apply(lambda row: [row['Latitude1'], row['Longitude1']], axis=1)
##Make a latLong column which contains an array which is the form required for map markers
torn_inc_map = folium.Map(location=[36.247802, -86.716847], zoom_start=10)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(torn_inc_map)
for location in tornado_incidents_df['LatLong']:
    folium.Marker(location=location).add_to(torn_inc_map)
torn_inc_map